<a href="https://colab.research.google.com/github/tonyw54/GeorgeBrown/blob/main/BERT_Misogyny_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


Copy the zip file at https://aclanthology.org/attachments/2021.eacl-main.114.Dataset.zip, then unzip it

In [ ]:
!wget https://aclanthology.org/attachments/2021.eacl-main.114.Dataset.zip
!unzip 2021.eacl-main.114.Dataset.zip

--2024-09-10 17:31:03--  https://aclanthology.org/attachments/2021.eacl-main.114.Dataset.zip
Resolving aclanthology.org (aclanthology.org)... 174.138.37.75
Connecting to aclanthology.org (aclanthology.org)|174.138.37.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2173869 (2.1M) [application/zip]
Saving to: ‘2021.eacl-main.114.Dataset.zip’

2021.eacl-main.114. 100%[===================>]   2.07M  1.21MB/s    in 1.7s    

2024-09-10 17:31:06 (1.21 MB/s) - ‘2021.eacl-main.114.Dataset.zip’ saved [2173869/2173869]

Archive:  2021.eacl-main.114.Dataset.zip
   creating: data/
  inflating: data/final_labels.csv   
  inflating: __MACOSX/data/._final_labels.csv  
  inflating: data/original_labels.csv  
  inflating: __MACOSX/data/._original_labels.csv  


In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
data = pd.read_csv("data/final_labels.csv")

Various types of text preprocessing, as done below, makes the model perform worse

In [ ]:
"""
Tried text preprocessing, but it is not useful for the bert-base-uncased model, which requires full sentences

import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Download stopwords
nltk.download('stopwords')

data = pd.read_csv("data/final_labels.csv")

# prompt: what data preprocessing can I use on the data["body"] field?

# Remove punctuation
data["body"] = data["body"].str.replace('[^\w\s]', '')

# Convert to lowercase
data["body"] = data["body"].str.lower()

# Tokenize the text (using a tokenizer from a pre-trained model)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Check for empty strings and handle them
# Check for empty strings and handle them
data["body"] = data["body"].apply(lambda x: tokenizer.tokenize(str(x)) if len(str(x)) > 0 else [])

# Remove stop words
stop_words = set(stopwords.words('english'))
data["body"] = data["body"].apply(lambda x: [word for word in x if word not in stop_words])

# Stemming
stemmer = PorterStemmer()
data["body"] = data["body"].apply(lambda x: [stemmer.stem(word) for word in x])
"""

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (689 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
data["body"]

,body
0,Do you have the skin of a 80 year old grandma?...
1,This is taking a grain of truth and extrapolat...
2,Honestly my favorite thing about this is that ...
3,Source? Doesnt sound right to me idk
4,"Damn, I saw a movie in which the old woman bat..."
...,...
6562,Should I send my ex-boyfriends mom a thank you...
6563,"1. Whatever vibration you put out, you will at..."
6564,"1. Whatever vibration you put out, you will at..."
6565,>Please keep in mind that just because YOU vie...


Tokenize the text using a pre-trained tokenizer. We will use the BERT classifier https://en.wikipedia.org/wiki/BERT_(language_model)#:~:text=Fine%2Dtuning,-Finetuned%20tasks%20for&BERT%20is%20meant%20as%20a,applications%20in%20natural%20language%20processing

In [ ]:
# Ensure all values in the "body" column are strings
data["body"] = data["body"].astype(str)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(data["body"].tolist(), padding=True, truncation=True, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
label_encoder = LabelEncoder()
data["level_1_encoded"] = label_encoder.fit_transform(data["level_1"])
labels = torch.tensor(data["level_1_encoded"].tolist())

In [ ]:
len(labels)

6567

Load a pre-trained BERT model for sequence classification:

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Assuming binary classification

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create a Hugging Face Dataset object
dataset = Dataset.from_pandas(data[["body", "level_1_encoded"]])

# Define a function to tokenize the input text and include labels
def tokenize_function(examples):
    # Remove labels from tokenizer call
    tokenized = tokenizer(examples["body"], padding="max_length", truncation=True)
    # Add labels to the tokenized output
    tokenized['labels'] = examples["level_1_encoded"]
    return tokenized

# Apply the tokenize_function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset into training and validation sets
train_dataset = tokenized_datasets.train_test_split(test_size=0.2)["train"]
eval_dataset = tokenized_datasets.train_test_split(test_size=0.2)["test"]

Map:   0%|          | 0/6567 [00:00<?, ? examples/s]

1. Generate base model
2. Use preprocessing to see if that helps
  - Preprocessing makes the model worse, since the bert-base-uncased model expects full sentences
3. Set up early stopping, since model training takes so long

In [ ]:
#Import EarlyStoppingCallback from transformers
from transformers import EarlyStoppingCallback

# Define the training arguments with early stopping
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch", # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Use eval_loss to determine the best model
    save_strategy = "epoch" # Save the model at the end of each epoch
)

# Create a Trainer instance with the early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop training after 2 epochs of increasing validation loss
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Fine-tune the model
# This will take a long time
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.147170
2,0.234800,0.097900


Epoch,Training Loss,Validation Loss
1,No log,0.147170
2,0.234800,0.097900
3,0.234800,0.072281


TrainOutput(global_step=987, training_loss=0.17263591808871778, metrics={'train_runtime': 1579.3098, 'train_samples_per_second': 9.978, 'train_steps_per_second': 0.625, 'total_flos': 4146367121418240.0, 'train_loss': 0.17263591808871778, 'epoch': 3.0})

In [ ]:
# Save the model created above, since it takes half an hour to build

trainer.save_model("./misogyny_classifier")

In [ ]:
# Import trained model from the misogyny_classifier folder

# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("./misogyny_classifier")

In [ ]:
def predict_label(text):
  inputs = tokenizer(text, return_tensors="pt").to('cuda') # Move inputs to the GPU
  with torch.no_grad():
    outputs = model(**inputs)
    # Get the predicted label
  predicted_class = torch.argmax(outputs.logits, dim=1).item()
  # Get the original label from the label encoder
  predicted_label = label_encoder.inverse_transform([predicted_class])[0]
  return predicted_label

In [ ]:
predict_label("I don't like red cars.")

'Nonmisogynistic'

In [ ]:
predict_label("Women are gold diggers.")

'Misogynistic'

In [ ]:
predict_label("You should go back to the kitchen.")

'Nonmisogynistic'

In [ ]:
predict_label("Women should go back to the kitchen.")

'Nonmisogynistic'

In [ ]:
predict_label("Marie Curie won two Nobel prizes in different categories. One in Physics and one in Chemistry. She is the only person to have ever done so.")

'Nonmisogynistic'

In [ ]:
# prompt: give me a number of suggestions for how I can optimize this model

# 1. Increase the dataset size:
#    - If possible, gather more data to improve the model's generalization ability.
#    - Consider data augmentation techniques to create variations of existing data.

# 2. Fine-tune with a larger BERT model:
#    - Explore models like "bert-large-uncased" for potentially better performance.
#    - Be aware that larger models require more computational resources.

# 3. Optimize hyperparameters:
#    - Experiment with different learning rates, batch sizes, and number of epochs.
#    - Use techniques like grid search or random search to find optimal values.

# 4. Improve data preprocessing:
#    - Consider more advanced text cleaning techniques (e.g., removing stop words, stemming, lemmatization).
#    - Explore different tokenization strategies (e.g., using subword tokenization).

# 5. Use a different model architecture:
#    - Explore other pre-trained models like RoBERTa, XLNet, or DistilBERT.
#    - Consider using different architectures like CNNs or RNNs for text classification.

# 6. Employ early stopping:
#    - Monitor the validation loss during training and stop training when it starts to increase.
#    - This can prevent overfitting and save training time.

# 7. Utilize learning rate scheduling:
#    - Implement learning rate scheduling (e.g., linear decay, cosine annealing) to improve convergence.

# 8.  Regularization:
#    - Add dropout layers to the model to prevent overfitting.
#    - Explore other regularization techniques like weight decay.

# 9.  Gradient accumulation:
#    - If you have memory constraints, use gradient accumulation to simulate larger batch sizes.

# 10.  Hardware optimization:
#    - Use a GPU with more memory for larger datasets and models.
#    - Consider using TPUs for even faster training.


In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MisogynyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).long() # Changed to long()
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
data["body"]

,body
0,Do you have the skin of a 80 year old grandma?...
1,This is taking a grain of truth and extrapolat...
2,Honestly my favorite thing about this is that ...
3,Source? Doesnt sound right to me idk
4,"Damn, I saw a movie in which the old woman bat..."
...,...
6562,Should I send my ex-boyfriends mom a thank you...
6563,"1. Whatever vibration you put out, you will at..."
6564,"1. Whatever vibration you put out, you will at..."
6565,>Please keep in mind that just because YOU vie...


In [ ]:
class MisogynyClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pool = nn.AvgPool1d(2)
        self.fc1 = nn.Linear(embedding_dim // 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        print(f"Input shape: {x.shape}")  # Check input shape
        out = self.embedding(x)
        print(f"After embedding: {out.shape}")  # Check shape after embedding
        out = out.permute(0, 2, 1)
        print(f"After permute: {out.shape}")  # Check shape after permute
        out = self.pool(out)
        print(f"After pool: {out.shape}")  # Check shape after pooling
        out = out.squeeze(1)
        print(f"After squeeze: {out.shape}")  # Check shape after squeeze
        out = self.fc1(out)
        print(f"After fc1: {out.shape}")  # Check shape after fc1
        out = self.fc2(out)
        print(f"After fc2: {out.shape}")  # Check shape after fc2
        return out

In [ ]:
!pip install transformers
import torch
from transformers import AutoTokenizer

# Assume the data is in a pandas DataFrame called 'data'
# with columns 'sentence' and 'label' (0 for non-misogynistic, 1 for misogynistic)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') # You can replace 'bert-base-uncased' with any model of your choice

In [ ]:
encodings = tokenizer([str(sentence) for sentence in data['body']], truncation=True, padding=True)

# Convert labels to a tensor
labels = torch.tensor(labels)

# Create the dataset and dataloader
dataset = MisogynyDataset(encodings, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

<ipython-input-32-4cdc56c14679>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [ ]:
model = MisogynyClassifier(vocab_size=512, embedding_dim=100, hidden_dim=256, output_dim=2) # Added embedding_dim
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Move model to the GPU
model.to('cuda')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

!pip install transformers
from transformers import AutoTokenizer


class MisogynyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


class MisogynyClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pool = nn.AvgPool1d(2)
        self.fc1 = nn.Linear(embedding_dim // 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Check if input is on the correct device (CPU or GPU)
        print(f"Input device: {x.device}")
        print(f"Input shape: {x.shape}")
        out = self.embedding(x)
        print(f"After embedding: {out.shape}")
        out = out.permute(0, 2, 1)
        print(f"After permute: {out.shape}")
        out = self.pool(out)
        print(f"After pool: {out.shape}")

        # Add a check to ensure the tensor is not empty after pooling
        if out.size(2) == 0:
            print("Warning: Pooling resulted in an empty tensor. Returning zeros.")
            return torch.zeros(out.size(0), out.size(1))

        out = out.squeeze(2)
        print(f"After squeeze: {out.shape}")
        out = self.fc1(out)
        print(f"After fc1: {out.shape}")
        out = self.fc2(out)
        print(f"After fc2: {out.shape}")
        return out


# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Example data (replace with your actual data)
data = {'body': ['This is an example sentence.', 'Another example sentence.']}
labels = [0, 1]

# Tokenize the input data
encodings = tokenizer([str(sentence) for sentence in data['body']], truncation=True, padding=True)

# Convert labels to a tensor
labels = torch.tensor(labels)

# Create the dataset and dataloader
dataset = MisogynyDataset(encodings, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)  # Reduced batch size to 2 for this example

# Instantiate the model with adjusted vocab_size
model = MisogynyClassifier(vocab_size=tokenizer.vocab_size, embedding_dim=100, hidden_dim=256, output_dim=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Move model to the GPU if available
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.to(device)
    print("Model moved to GPU.")
else:
    device = torch.device('cpu')
    print("No GPU available, using CPU.")

# Example training loop (replace with your actual training loop)
for epoch in range(num_epochs):
   for batch in dataloader:
     print("Batch loaded")  # Check if batches are loaded
     input_ids = batch['input_ids']
     attention_mask = batch['attention_mask']
     labels = batch['labels']

     optimizer.zero_grad()

     # Move input_ids and labels to the GPU - No need to add an extra dimension
     input_ids = input_ids.long().to('cuda')
     labels = labels.to('cuda')

     outputs = model(input_ids)
     loss = loss_fn(outputs, labels)
     loss.backward()
     optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
!nvidia-smi

Tue Sep 10 17:47:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              33W /  70W |    141MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
num_epochs = 10  # Adjust as needed

for epoch in range(num_epochs):
   for batch in dataloader:
     print("Batch loaded")  # Check if batches are loaded
     input_ids = batch['input_ids']
     attention_mask = batch['attention_mask']
     labels = batch['labels']

     optimizer.zero_grad()

     # Move input_ids and labels to the GPU - No need to add an extra dimension
     input_ids = input_ids.long().to('cuda')
     labels = labels.to('cuda')

     outputs = model(input_ids)
     loss = loss_fn(outputs, labels)
     loss.backward()
     optimizer.step()

Batch loaded


<ipython-input-9-5bfa8f64aad6>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx]).long() # Changed to long()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
text = "I don't like red cars."
inputs = tokenizer(text, return_tensors="pt").to('cuda') # Move inputs to the GPU
print (inputs)
with torch.no_grad():
  # Pass input_ids directly as a positional argument
  outputs = model(inputs['input_ids'].float()) # Convert input_ids to Float
  # Get the predicted label
predicted_class = torch.argmax(outputs, dim=1).item() # Remove .logits
# Get the original label from the label encoder
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

{'input_ids': tensor([[ 101, 1045, 2123, 1005, 1056, 2066, 2417, 3765, 1012,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 512x768)

In [ ]:
predict_label("I don't like red cars.")

{'input_ids': tensor([[ 101, 1045, 2123, 1005, 1056, 2066, 2417, 3765, 1012,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 512x768)

In [ ]:
def predict_label(text):
  inputs = tokenizer(text, return_tensors="pt").to('cuda')
  print(inputs)  # Print the tokenizer output
  with torch.no_grad():
    outputs = model(inputs['input_ids'].flatten().float())
  predicted_class = torch.argmax(outputs, dim=1).item()
  predicted_label = label_encoder.inverse_transform([predicted_class])[0]
  return predicted_label